In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_16_9_0,0.999705,0.998693,0.999567,0.999101,0.035546,0.176739,0.049990,0.117093,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
1,model_16_9_1,0.999705,0.998692,0.999567,0.999100,0.035566,0.176846,0.050010,0.117159,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
2,model_16_9_2,0.999705,0.998692,0.999567,0.999100,0.035573,0.176883,0.050016,0.117181,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
3,model_16_9_3,0.999705,0.998692,0.999567,0.999100,0.035574,0.176890,0.050016,0.117185,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
4,model_16_9_5,0.999705,0.998692,0.999567,0.999100,0.035575,0.176896,0.050016,0.117187,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
...,...,...,...,...,...,...,...,...,...,...
1092,model_12_7_1,0.954312,0.969916,0.994568,0.988090,5.510007,1.697386,0.957259,1.349093,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1093,model_6_8_1,0.954137,0.967878,0.995723,0.977408,5.531062,5.285598,0.384151,2.979034,"Hidden Size=[20], regularizer=0.1, learning_ra..."
1104,model_15_9_3,0.953032,0.938801,0.979753,0.975213,5.664342,2.647217,2.371512,2.517473,"Hidden Size=[30, 15], regularizer=0.01, learni..."
1108,model_13_8_7,0.952405,0.981841,0.960482,0.971843,5.739919,2.779801,5.999055,4.294745,"Hidden Size=[10, 5], regularizer=0.01, learnin..."
